In [25]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import pdfplumber
import docx
import win32com.client
from langchain.schema import Document

In [26]:
import os
file_path = "./docs/包装机械的CE认证.pdf"
file_extension = os.path.splitext(file_path)[-1].lower()
print(f"string:{file_extension}")

string:.pdf


In [27]:
def extract_docs(folder_path):
    paths = []
    for root, dir, files in os.walk(folder_path):
        for f in files:
            f_path = os.path.join(root, f)
            paths.append(f_path)
    return paths
pa = extract_docs("./docs")
print(pa)

['./docs\\YCT 549.8-2016 烟草机械 烟草专用机械鉴别检验规程 第8部分：烟用包装机械.pdf', './docs\\包装机械的CE认证.pdf', './docs\\包装机械结构与设计.pdf', './docs\\包装机械设计.pdf', './docs\\包装机械选用手册.pdf', './docs\\包装机械选用手册上-印刷实务.pdf', './docs\\包装机械选用手册下-印刷实务.pdf', './docs\\印刷机械包装机械配套产品实用手册.pdf', './docs\\工-机械全书1-2.pdf', './docs\\工-机械全书3-4.pdf', './docs\\工-机械全书5-6.pdf', './docs\\工-机械全书7-11.pdf', './docs\\工-机械最优化设计及应用实例1.pdf', './docs\\工-机械设计(清华精仪).pdf', './docs\\工-机械设计(西工大).pdf', './docs\\教-机 械 制 造 基 础1.doc', './docs\\教-机 械 制 造 基 础1.docx', './docs\\教-机械-数学和力学基础知识.doc', './docs\\教-机械-数学和力学基础知识.docx', './docs\\教-机械制图.doc', './docs\\教-机械制图.docx', './docs\\教-机械制造工艺基础.doc', './docs\\教-机械制造工艺基础.docx', './docs\\教-机械制造概论.doc', './docs\\教-机械制造概论.docx', './docs\\教-机械工程材料学.doc', './docs\\教-机械工程材料学.docx', './docs\\教-机械现代控制工程.doc', './docs\\教-机械现代控制工程.docx', './docs\\教-机械电子学.doc', './docs\\教-机械电子学.docx', './docs\\教-机械零件与原理.doc', './docs\\教-机械零件与原理.docx', './docs\\钢管的验收、包装、标志和质量证明书 .pdf']


In [28]:
def extract_text(path):# 某一个path如：./docs/包装机械结构与设计.pdf
    text=''
    file_extension = os.path.splitext(path)[-1].lower()
    if file_extension == '.pdf':
        with pdfplumber.open(path) as f:
            for page in f.pages:
                text+=page.extract_text()
    elif file_extension == '.docs':
        file_path = convert_doc_to_docx(path, f"{os.path.splitext(path)[0]}.docx")
        docf = docx.Document(file_path)
        for paragraph in docf.paragraphs:
            text+=paragraph.text

In [29]:
for i in pa:
    print(i)

./docs\YCT 549.8-2016 烟草机械 烟草专用机械鉴别检验规程 第8部分：烟用包装机械.pdf
./docs\包装机械的CE认证.pdf
./docs\包装机械结构与设计.pdf
./docs\包装机械设计.pdf
./docs\包装机械选用手册.pdf
./docs\包装机械选用手册上-印刷实务.pdf
./docs\包装机械选用手册下-印刷实务.pdf
./docs\印刷机械包装机械配套产品实用手册.pdf
./docs\工-机械全书1-2.pdf
./docs\工-机械全书3-4.pdf
./docs\工-机械全书5-6.pdf
./docs\工-机械全书7-11.pdf
./docs\工-机械最优化设计及应用实例1.pdf
./docs\工-机械设计(清华精仪).pdf
./docs\工-机械设计(西工大).pdf
./docs\教-机 械 制 造 基 础1.doc
./docs\教-机 械 制 造 基 础1.docx
./docs\教-机械-数学和力学基础知识.doc
./docs\教-机械-数学和力学基础知识.docx
./docs\教-机械制图.doc
./docs\教-机械制图.docx
./docs\教-机械制造工艺基础.doc
./docs\教-机械制造工艺基础.docx
./docs\教-机械制造概论.doc
./docs\教-机械制造概论.docx
./docs\教-机械工程材料学.doc
./docs\教-机械工程材料学.docx
./docs\教-机械现代控制工程.doc
./docs\教-机械现代控制工程.docx
./docs\教-机械电子学.doc
./docs\教-机械电子学.docx
./docs\教-机械零件与原理.doc
./docs\教-机械零件与原理.docx
./docs\钢管的验收、包装、标志和质量证明书 .pdf


In [30]:
def convert_doc_to_docx(input_path, output_path):
    input_path = os.path.abspath(input_path)
    output_path = os.path.abspath(output_path)

    try:
        word = win32com.client.Dispatch("Word.Application")
        word.Visible = False  # 隐藏 Word 窗口
        doc = word.Documents.Open(input_path)
        doc.SaveAs(output_path, FileFormat=16)  # 16 表示 .docx 格式
        doc.Close()
        word.Quit()
        print(f"Successfully converted {input_path} to {output_path}")
        return output_path
    except Exception as e:
        print(f"Error: {e}")

In [31]:
path = "docs/教-机械制图.doc"
convert_doc_to_docx(path, f"{os.path.splitext(path)[0]}.docx")

Successfully converted D:\codes\python_project\LangChain_Chat\chatRobot\docs\教-机械制图.doc to D:\codes\python_project\LangChain_Chat\chatRobot\docs\教-机械制图.docx


'D:\\codes\\python_project\\LangChain_Chat\\chatRobot\\docs\\教-机械制图.docx'

In [32]:
f = docx.Document("./docs/教-机械零件与原理.docx")
for p in f.paragraphs:
    print(p.text)

In [33]:
f = os.path.splitext('docs/教-机械零件与原理.docx')
f

('docs/教-机械零件与原理', '.docx')

In [34]:
for path, dirs, files in os.walk("docs"):
    for f in files:
        print(f)
        if f.endswith(".pdf"):
            print("pdf")
        elif f.endswith(".docs"):
            print("docs")

YCT 549.8-2016 烟草机械 烟草专用机械鉴别检验规程 第8部分：烟用包装机械.pdf
pdf
包装机械的CE认证.pdf
pdf
包装机械结构与设计.pdf
pdf
包装机械设计.pdf
pdf
包装机械选用手册.pdf
pdf
包装机械选用手册上-印刷实务.pdf
pdf
包装机械选用手册下-印刷实务.pdf
pdf
印刷机械包装机械配套产品实用手册.pdf
pdf
工-机械全书1-2.pdf
pdf
工-机械全书3-4.pdf
pdf
工-机械全书5-6.pdf
pdf
工-机械全书7-11.pdf
pdf
工-机械最优化设计及应用实例1.pdf
pdf
工-机械设计(清华精仪).pdf
pdf
工-机械设计(西工大).pdf
pdf
教-机 械 制 造 基 础1.doc
教-机 械 制 造 基 础1.docx
教-机械-数学和力学基础知识.doc
教-机械-数学和力学基础知识.docx
教-机械制图.doc
教-机械制图.docx
教-机械制造工艺基础.doc
教-机械制造工艺基础.docx
教-机械制造概论.doc
教-机械制造概论.docx
教-机械工程材料学.doc
教-机械工程材料学.docx
教-机械现代控制工程.doc
教-机械现代控制工程.docx
教-机械电子学.doc
教-机械电子学.docx
教-机械零件与原理.doc
教-机械零件与原理.docx
钢管的验收、包装、标志和质量证明书 .pdf
pdf
